# Markov Partition Construction Algorithm - Chapter 4.2

In [ ]:
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from scipy import optimize
import shapely
from shapely.geometry import LineString, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon
from typing import Union, Optional
from experimental.utils.dynamic_system import DynamicSystem
from experimental.utils.partition import Partition

## Define System's Dynamics in Euclidean Plane and on Torus

In [ ]:
A = np.array([[1,1], [1,0]])
A

In [ ]:
def phi(x: np.array) -> np.array:
    return np.dot(A, x) % 1

In [ ]:
def d_phi(x: np.array):
    return np.transpose(A)

## Compute Eigendecomposition

In [ ]:
# get eigenvalues and right eigenvectors, i.e. transposed eigenvectors
eig_vals, eig_vects = np.linalg.eig(A)
# retrieve eigenvectors from right eigenvectors
eig_vects = np.transpose(eig_vects)

In [ ]:
print(f"Eigenvalues: {eig_vals}")

In [ ]:
print(f"Eigenvectors: {eig_vects}")

## Analysis of Hyperbolicity and Invertibility

In [ ]:
def is_hyperbolic(eig_vals: np.array) -> bool:
    contr_seg = any(np.abs(eig_val) < 1 for eig_val in eig_vals)
    exp_seg = any(np.abs(eig_val) > 1 for eig_val in eig_vals)

    return contr_seg and exp_seg

In [ ]:
system_hyperbolic = is_hyperbolic(eig_vals)
print(f"Dynamic system is hyperbolic: {system_hyperbolic}")

In [ ]:
def is_invertible(A: np.array) -> bool:
    return np.abs(np.linalg.det(A)) == 1

In [ ]:
system_invertible = is_invertible(A)
print(f"Dynamic system is invertble: {system_invertible}")

## Computation of Bases for Contracting and Expanding Segments 

In [ ]:
def expanding_segments(eig_vals: np.array, eig_vects: np.array) -> np.array:
    exp_eig_spaces_msk = np.abs(eig_vals) > 1
    print(exp_eig_spaces_msk)
    return eig_vects[exp_eig_spaces_msk]

In [ ]:
def contracting_segments(eig_vals: np.array, eig_vects: np.array) -> np.array:
    contr_eig_spaces_msk = np.abs(eig_vals) < 1
    print(contr_eig_spaces_msk)
    return eig_vects[contr_eig_spaces_msk]

In [ ]:
contr_segs = contracting_segments(eig_vals, eig_vects)
print(f"Contracting segments: {contr_segs}")

In [ ]:
exp_segs = expanding_segments(eig_vals, eig_vects)
print(f"Expanding segments: {exp_segs}")

## Translate between Euclidea Plane and Torus

In [ ]:
def q(x: np.array) -> np.array:
    return x % 1

In [ ]:
# sample point in R^2 between -10 and 10
x = np.random.uniform(low=-10, high=10, size=2)

In [ ]:
# apply system's dynamics A in plane, then push down to torus by q
plane_dynamics_result = q(np.dot(A, x))
print(f"Next point on torus: {plane_dynamics_result}")

In [ ]:
# push down point to torus, then apply system's dynamics phi on torus
torus_dynamics_result = phi(q(x))
print(f"Next point on torus: {torus_dynamics_result}")

In [ ]:
if np.all(plane_dynamics_result == torus_dynamics_result):
    print(f"System dynamics in plane and on torus commute by q!")
else:
    print(f"System dynamics in plane and on torus do not commute by q!")

## Calculate Inetrsections between Lines

In [ ]:
def perp(a) :
    b = np.empty_like(a)
    b[0] = -a[1]
    b[1] = a[0]
    return b

# line segment a given by endpoints a1, a2
# line segment b given by endpoints b1, b2
def seg_intersect(a, b) :
    a1, a2 = a
    b1, b2 = b

    da = a2 - a1
    db = b2 - b1
    dp = a1 - b1
    dap = perp(da)
    denom = np.dot(dap, db)
    num = np.dot(dap, dp)

    return (num / denom.astype(float))*db + b1

## Calculate Hyperbolic Fixed Point

In [ ]:
# reformulate fixed point problem Ax = x as root finding problem Ax-x = 0
def objective_func(x: np.array) -> np.array:
    return np.dot(A, x) - x

In [ ]:
# function that calculates jacobian of objective function, namely D(Ax-x) = A-I
def jac(x: np.array) -> np.array:
    return A - np.identity(n=2)

In [ ]:
# compute fixed point
# find roots of objective function with Levenberg-Marquardt algorithm
def compute_fixed_point(objective_func: callable, jac: callable, init_x: np.array=np.array([1,1])) -> np.array:
    root_sol = optimize.root(objective_func, init_x, jac=jac, method="lm")
    if not root_sol.success:
        print(f"Failed to find a fixed point. Error: {root_sol.message}")
        return None

    fixed_point = root_sol.x
    return fixed_point

In [ ]:
def fixed_point_is_hyperbolic(fixed_point: np.array, jac: callable) -> bool:
    eig_vals = np.linalg.eigvals(jac(fixed_point))
    return np.all(eig_vals != 1)

In [ ]:
fixed_point = compute_fixed_point(objective_func, jac, np.array([1,2]))
print(f"Fixed point: {fixed_point}")
if fixed_point_is_hyperbolic(fixed_point, d_phi):
    print(f"Fixed point {fixed_point} is a hyperbolic one!")
else:
    print(f"Fixed point {fixed_point} is not a hyperbolic one!")

## Calculate Symmetric (un)stable Branches around Hyperbolic Fixed Point

In [ ]:
r = 0.5

In [ ]:
vs = eig_vects[np.abs(eig_vals) < 1][0]
vu = eig_vects[np.abs(eig_vals) > 1][0]
print(f"W^u: {vu}")
print(f"W^s: {vs}")

In [ ]:
x1 = q(fixed_point + r * vs)
x2 = q(fixed_point - r * vs)
print(f"x1: {x1}")
print(f"x2: {x2}")

In [ ]:
y1 = q(fixed_point + r * vu)
y2 = q(fixed_point - r * vu)
print(f"y1: {y1}")
print(f"y2: {y2}")

In [ ]:
unit_square = np.array([[0,0], [0,1], [1,1], [1,0], [0,0]])
points = np.array([fixed_point, x1, x2, y1, y2])

w_x1 = np.array([[1,0],x1])
w_x2 = np.array([[0,1],x2])

w_y1 = np.array([[0,0],y1])
w_y2 = np.array([[1,1],y2])

fig, ax = plt.subplots()
ax.plot(unit_square[:, 0], unit_square[:, 1], "g")

ax.plot(w_x1[:,0], w_x1[:,1], "r")
ax.plot(w_x2[:,0], w_x2[:,1], "r")
ax.plot(w_y1[:,0], w_y1[:,1], "b")
ax.plot(w_y2[:,0], w_y2[:,1], "b")

ax.scatter(points[:, 0], points[:, 1])
labels = ["z*", "x1", "x2", "y1", "y2"]

for i, txt in enumerate(labels):
    ax.annotate(txt, (points[i, 0] + 0.02, points[i, 1] + 0.02))

## Trace Stable and Unstable Branches Analytically

In [ ]:
def curve_intersections(z0: np.array, z1: np.array, v0: np.array, v1: np.array):
    """
    Compute the point in time of intersection between two linear parameterized curves.
    Args:
        z0 (np.array): origin of first curve
        z1 (np.array): origin of second curve
        v0 (np.array): direction vector of first curve
        v1 (np.array): direction vector of second curve

    Returns:
        t (float): time when first curve meets intersection point
        s (float): time when second curve meets intersection point
        p (np.array): intersection point
    """
    t = np.dot(v0, z1-z0)
    s = np.dot(v1, z0-z1)

    p = z0 + t * v0

    assert np.allclose(p, z1 + s * v1, atol=1e-08), "q(t) != r(s), something went wrong."

    return t, s, p

In [ ]:
u1 = np.array([0,0])
u2 = np.array([0,1])
u3 = np.array([1,1])
u4 = np.array([1,0])

In [ ]:
t1, s1, p1 = curve_intersections(u1, u2, vu, vs)

print(f"Time of intersection for W^u: {np.abs(t1)}")
print(f"Time of intersection for W^s: {np.abs(s1)}")

if np.abs(t1) > np.abs(s1):
    print("W^u ends onto W^s")
else:
    print("W^s ends onto W^u")

print(f"Parametrically calculated point of intersection: {p1}")

In [ ]:
t2, s2, p2 = curve_intersections(u1, u4, vu, vs)

print(f"Time of intersection for W^u: {np.abs(t2)}")
print(f"Time of intersection for W^s: {np.abs(s2)}")

if np.abs(t2) > np.abs(s2):
    print("W^u ends onto W^s")
else:
    print("W^s ends onto W^u")

print(f"Parametrically calculated point of intersection: {p2}")

In [ ]:
t3, s3, p3 = curve_intersections(u3, u4, vu, vs)

print(f"Time of intersection for W^u: {np.abs(t3)}")
print(f"Time of intersection for W^s: {np.abs(s3)}")

if np.abs(t3) > np.abs(s3):
    print("W^u ends onto W^s")
else:
    print("W^s ends onto W^u")

print(f"Parametrically calculated point of intersection: {p3}")

In [ ]:
t4, s4, p4 = curve_intersections(u3, u2, vu, vs)

print(f"Time of intersection for W^u: {np.abs(t4)}")
print(f"Time of intersection for W^s: {np.abs(s4)}")

if np.abs(t4) > np.abs(s4):
    print("W^u ends onto W^s")
else:
    print("W^s ends onto W^u")

print(f"Parametrically calculated point of intersection: {p4}")

In [ ]:
print(f"First eigenvector: ({eig_vects[0,0]}, {eig_vects[0,1]})")

In [ ]:
u = np.array([u1, u2, u3, u4, u1])
p = np.array([p1, p2, p3, p4])

P1 = np.array([p1, p2, p3, p4, p1])

P2a = np.array([u1, p2, u4])
P2b = np.array([u2, p4, u3])

P3a = np.array([u1, p1, u2])
P3b = np.array([u3, p3, u4])

#draw unit square
plt.plot(u[:, 0], u[:, 1], "r")

# draw rectangle P1
plt.plot(P1[:, 0], P1[:, 1], "b-")

# draw rectangle P2
plt.plot(P2a[:, 0], P2a[:, 1], "b-")
plt.plot(P2b[:, 0], P2b[:, 1], "b-")

# draw rectangle P3
plt.plot(P3a[:, 0], P3a[:, 1], "b-")
plt.plot(P3b[:, 0], P3b[:, 1], "b-")

# draw rectangle vertices
plt.plot(p[:, 0], p[:, 1], "yo")
plt.plot(u[:, 0], u[:, 1], "yo")

plt.show()

In [ ]:
P1_phi = np.array([phi(p1), phi(p2), phi(p3), phi(p4)])

phi_p1_a = seg_intersect(np.array([u3,u4]), np.array([phi(p1), phi(p1)+vu]))
phi_p2_a = seg_intersect(np.array([u1,u2]), np.array([phi(p2), phi(p2)-vu]))
phi_p3_a = seg_intersect(np.array([u1,u2]), np.array([phi(p3), phi(p3)-vu]))
phi_p4_a = seg_intersect(np.array([u3,u4]), np.array([phi(p4), phi(p4)+vu]))

P1_phi_boundaries_a = np.array([phi(p1), phi_p1_a, phi_p4_a, phi(p4)])
P1_phi_boundaries_b = np.array([phi_p3_a, phi_p2_a, phi(p2), phi(p3)])

In [ ]:
u = np.array([u1, u2, u3, u4, u1])
p = np.array([p1, p2, p3, p4])

P1 = np.array([p1, p2, p3, p4, p1])

P2a = np.array([u1, p2, u4])
P2b = np.array([u2, p4, u3])

P3a = np.array([u1, p1, u2])
P3b = np.array([u3, p3, u4])

#draw unit square
plt.plot(u[:, 0], u[:, 1], "r")

# draw rectangle P1
plt.plot(P1[:, 0], P1[:, 1], "b-")

# draw rectangle P2
plt.plot(P2a[:, 0], P2a[:, 1], "b-")
plt.plot(P2b[:, 0], P2b[:, 1], "b-")

# draw rectangle P3
plt.plot(P3a[:, 0], P3a[:, 1], "b-")
plt.plot(P3b[:, 0], P3b[:, 1], "b-")

# draw P1-phi rectangle vertices
plt.plot(P1_phi[:, 0], P1_phi[:, 1], "yo")

# draw P1-phi rectangle area
plt.fill(P1_phi_boundaries_a[:, 0], P1_phi_boundaries_a[:, 1], "gray", alpha=0.3)
plt.fill(P1_phi_boundaries_b[:, 0], P1_phi_boundaries_b[:, 1], "gray", alpha=0.3)

plt.show()

In [ ]:
P2_phi = np.array([phi(u1)+np.array([1,0]), phi(p2), phi(u4)+np.array([0,1]), phi(p4)])

phi_p1_a = seg_intersect(np.array([u3,u4]), np.array([phi(p1), phi(p1)+vu]))
phi_p2_a = seg_intersect(np.array([u1,u2]), np.array([phi(p2), phi(p2)-vu]))
phi_p3_a = seg_intersect(np.array([u1,u2]), np.array([phi(p3), phi(p3)-vu]))
phi_p4_a = seg_intersect(np.array([u3,u4]), np.array([phi(p4), phi(p4)+vu]))

P2_phi_boundaries_a = np.array([u2, phi(p2), phi_p2_a])
P2_phi_boundaries_b = np.array([u4, phi(p4), phi_p4_a])
P2_phi_boundaries_c = np.array([u3, phi_p3_a, u1, phi_p1_a])

In [ ]:
u = np.array([u1, u2, u3, u4, u1])
p = np.array([p1, p2, p3, p4])

P1 = np.array([p1, p2, p3, p4, p1])

P2a = np.array([u1, p2, u4])
P2b = np.array([u2, p4, u3])

P3a = np.array([u1, p1, u2])
P3b = np.array([u3, p3, u4])

#draw unit square
plt.plot(u[:, 0], u[:, 1], "r")

# draw rectangle P1
plt.plot(P1[:, 0], P1[:, 1], "b-")

# draw rectangle P2
plt.plot(P2a[:, 0], P2a[:, 1], "b-")
plt.plot(P2b[:, 0], P2b[:, 1], "b-")

# draw rectangle P3
plt.plot(P3a[:, 0], P3a[:, 1], "b-")
plt.plot(P3b[:, 0], P3b[:, 1], "b-")

# draw P2-phi rectangle vertices
plt.plot(P2_phi[:, 0], P2_phi[:, 1], "yo")

# draw P2-phi rectangle area
plt.fill(P2_phi_boundaries_a[:, 0], P2_phi_boundaries_a[:, 1], "gray", alpha=0.3)
plt.fill(P2_phi_boundaries_b[:, 0], P2_phi_boundaries_b[:, 1], "gray", alpha=0.3)
plt.fill(P2_phi_boundaries_c[:, 0], P2_phi_boundaries_c[:, 1], "gray", alpha=0.3)

plt.show()

In [ ]:
P3_phi = np.array([phi(u1), phi(p1), phi(u2)+np.array([1,0]), phi(p3)])

P3_phi_boundaries_a = np.array([u1, phi(p1), u4])
P3_phi_boundaries_b = np.array([u2, phi(p3), u3])

In [ ]:
u = np.array([u1, u2, u3, u4, u1])
p = np.array([p1, p2, p3, p4])

P1 = np.array([p1, p2, p3, p4, p1])

P2a = np.array([u1, p2, u4])
P2b = np.array([u2, p4, u3])

P3a = np.array([u1, p1, u2])
P3b = np.array([u3, p3, u4])

#draw unit square
plt.plot(u[:, 0], u[:, 1], "r")

# draw rectangle P1
plt.plot(P1[:, 0], P1[:, 1], "b-")

# draw rectangle P2
plt.plot(P2a[:, 0], P2a[:, 1], "b-")
plt.plot(P2b[:, 0], P2b[:, 1], "b-")

# draw rectangle P3
plt.plot(P3a[:, 0], P3a[:, 1], "b-")
plt.plot(P3b[:, 0], P3b[:, 1], "b-")

# draw P3-phi rectangle vertices
plt.plot(P3_phi[:, 0], P3_phi[:, 1], "yo")

# draw P3-phi rectangle area
plt.fill(P3_phi_boundaries_a[:, 0], P3_phi_boundaries_a[:, 1], "gray", alpha=0.3)
plt.fill(P3_phi_boundaries_b[:, 0], P3_phi_boundaries_b[:, 1], "gray", alpha=0.3)

plt.show()

## Approximate Stable and Unstable Branches Numerically

## Build our linear example

In [ ]:
A = np.array([[1,1], [1,0]])
default_delta = 10e-3
num_iters = 2
m_id = 1

In [ ]:
def phi_non_identified(x: np.array) -> np.array:
    return np.dot(A, x)

In [ ]:
def d_phi(x: np.array) -> np.array:
    return np.transpose(A)

In [ ]:
dynamic_system = DynamicSystem(phi_non_identified, d_phi, m_id)
partition = Partition(dynamic_system)

In [ ]:
branches, intersection_points = partition.compute_partition(num_iters, delta=default_delta)

In [ ]:
partition.plot_partition()

## Check intersection property for our linear example

In [ ]:
dist_delta = 10e-8

In [ ]:
p1 = intersection_points[0]
p2 = intersection_points[2]
p3 = intersection_points[3]
p4 = intersection_points[1]
P1 = np.array([p1, p2, p3, p4])

In [ ]:
phi_P1 = np.transpose(dynamic_system.phi(np.transpose(P1)))
phi_P1

In [ ]:
left_top = phi_P1[1]
left_bottom = phi_P1[0]
right_top = phi_P1[2]
right_bottom = phi_P1[3]

In [ ]:
left = [[left_bottom, left_bottom]]
right = [[right_bottom, right_bottom]]
top = [[left_top, left_top]]
bottom = [[left_bottom, left_bottom]]

In [ ]:
last_top = left_top
while Point(right_top).distance(MultiLineString(top)) > dist_delta:
        next_top = partition.wt_u(last_top, reverse_direction=False)
        if not dynamic_system.identification_occurs(next_top):
            top[-1].append(next_top)
        else:
            next_top = next_top % dynamic_system.m_id
            top.append([next_top, next_top])
        last_top = next_top

In [ ]:
MultiLineString(top)

In [ ]:
last_bottom = left_bottom
while Point(right_bottom).distance(MultiLineString(bottom)) > dist_delta:
        next_bottom = partition.wt_u(last_bottom, reverse_direction=False)
        if not dynamic_system.identification_occurs(next_bottom):
            bottom[-1].append(next_bottom)
        else:
            next_bottom = next_bottom % dynamic_system.m_id
            bottom.append([next_bottom, next_bottom])
        last_bottom = next_bottom

In [ ]:
MultiLineString(bottom)

In [ ]:
last_left = left_bottom
while Point(left_top).distance(MultiLineString(left)) > dist_delta:
        next_left = partition.wt_s(last_left, reverse_direction=False)
        if not dynamic_system.identification_occurs(next_left):
            left[-1].append(next_left)
        else:
            next_left = next_left % dynamic_system.m_id
            left.append([next_left, next_left])
        last_left = next_left

In [ ]:
MultiLineString(left)

In [ ]:
last_right = right_bottom
while Point(right_top).distance(MultiLineString(right)) > dist_delta:
        next_right = partition.wt_s(last_right, reverse_direction=False)
        if not dynamic_system.identification_occurs(next_right):
            right[-1].append(next_right)
        else:
            next_right = next_right % dynamic_system.m_id
            right.append([next_right, next_right])
        last_right = next_right

In [ ]:
MultiLineString(right)

In [ ]:
poly_left_points = [left[0], bottom[0], top[0]]
poly_left = MultiLineString(poly_left_points).convex_hull
poly_left

In [ ]:
poly_right_points = [bottom[1], top[1], right[0]]
poly_right = MultiLineString(poly_right_points).convex_hull
poly_right

In [ ]:
MultiPolygon([poly_left, poly_right]).intersection(MultiPolygon([poly_left, poly_right]))

In [ ]:
p1 = intersection_points[3]
p2 = intersection_points[5]
p3 = intersection_points[0]
p4 = intersection_points[6]
P2 = np.array([p1, p2, p3, p4])

In [ ]:
unit_square = np.array([[0, 0], [0, m_id], [m_id, m_id], [m_id, 0], [0, 0]])
plt.plot(unit_square[:, 0], unit_square[:, 1], "r-")
plt.plot(P1[:, 0], P1[:, 1], "go")
plt.plot(phi_P1[:, 0], phi_P1[:, 1], "co")
plt.plot(P2[:, 0], P2[:, 1], "yo")

In [ ]:
A

In [ ]:
np.linalg.eigvals(A)

In [ ]:
T = np.array([[0,0,1], [1,0,1], [0,1,0]])
T

In [ ]:
np.linalg.eigvals(T)

## Build linear example from 1985 paper

In [ ]:
A = np.array([[1,1], [1,2]])
default_delta = 10e-3
num_iters = 2
m_id = 1

In [ ]:
def phi_non_identified(x: np.array) -> np.array:
    return np.dot(A, x)

In [ ]:
def d_phi(x: np.array) -> np.array:
    return np.transpose(A)

In [ ]:
dynamic_system = DynamicSystem(phi_non_identified, d_phi, m_id)
partition = Partition(dynamic_system)

In [ ]:
branches, intersection_points = partition.compute_partition(num_iters, delta=default_delta)

In [ ]:
partition.plot_partition()

## Build our nonlinear example

In [ ]:
A = np.array([[1,1], [1,0]])
default_eps = 0.7
default_delta = 10e-3
num_iters = 2
m_id = 2*np.pi

In [ ]:
def phi_non_identified(x: np.array, eps: float=default_eps) -> np.array:
    return np.dot(A, x) + np.array([eps*np.cos(x[0]), 0*eps*np.cos(x[0])])

In [ ]:
def d_phi(x: np.array, eps: float=default_eps) -> np.array:
    x12, x22 = A[1]
    x11 = A[0,0] - eps*np.sin(x[0])
    x21 = A[1,0] - 0*eps*np.sin(x[0])
    return np.array([[x11, x12], [x21, x22]])

In [ ]:
dynamic_system = DynamicSystem(phi_non_identified, d_phi, m_id)
partition = Partition(dynamic_system)

In [ ]:
branches, intersection_points = partition.compute_partition(num_iters, delta=default_delta)

In [ ]:
partition.plot_partition()

## Build nonlinear example from 1985 paper

In [ ]:
A = np.array([[1,1], [1,2]])
default_eps = 0.7
default_delta = 10e-3
m_id = 2*np.pi
num_iters = 2

In [ ]:
def phi_non_identified(x: np.array, eps: float=default_eps) -> np.array:
    return np.dot(A, x) + np.array([eps*np.cos(x[0]), 2*eps*np.cos(x[0])])

In [ ]:
def d_phi(x: np.array, eps: float=default_eps) -> np.array:
    x12, x22 = A[1]
    x11 = A[0,0] - eps*np.sin(x[0])
    x21 = A[1,0] - 2*eps*np.sin(x[0])
    return np.array([[x11, x12], [x21, x22]])

In [ ]:
dynamic_system = DynamicSystem(phi_non_identified, d_phi, m_id)
partition = Partition(dynamic_system)

In [ ]:
branches, intersection_points = partition.compute_partition(num_iters, delta=default_delta)

In [ ]:
partition.plot_partition()